# Nationalhymnen: Aggregieren und Aufbereiten der Daten 

### Ressourcen: 
* [Wikipedia](https://de.wikipedia.org/Liste_der_Nationalhymnen): In der Wikipedia sind fast alle Nationalhymnen erfasst -- oftmals inklusive einer Übersetzung in die deutsche Sprache. Da die Liedtexte nicht einheitlich strukturiert sind, ist beim Auslesen Handarbeit nötig. 
* [Midi-Datenbank](https://www.kaggle.com/awesomepgm/national-anthems-of-every-country): Datenbank mit allen Midi-Dateien. 
* [Nationalanthems.info](http://www.nationalanthems.info/Texte): In der Sammlung von Nationalanthems.info sind Hintergrundinformationen zu Nationalhymnen erfasst. Verfügbar ist auch der Text sowie teilweise Übersetzungen ins Englische.

In [124]:
# Fürs Herunterladen der Rohdaten aus der Wikipedia. 
import wikipedia
wikipedia.set_lang("de")
from bs4 import BeautifulSoup
from requests import get
from time import sleep

# Für die Datenaufbereitung. 
import re
from langdetect import detect
from os import system
import pandas as pd

In [125]:
# Die Übersicht über die Nationalhymnen aus der Wikipedia wird geladen. 
r = wikipedia.page('Liste der Nationalhymnen')
s = BeautifulSoup(r.html(), 'lxml')
t = s.find('table', {'class': 'wikitable'})
tr = t.find_all('tr')

In [126]:
# Zusammenstellen der Links zu den Wikipedia-Seiten der Nationalhymnen. 
link_dict = dict()
for t in tr[1:]: 
    tds = t.find_all('td')
    land = tds[0].find('a')['title']
    link_song = 'https://de.wikipedia.org' + tds[1].find('a')['href']
    link_dict[land] = link_song

In [4]:
# Herunterladen der Texte der Nationalhymnen. 
lyrics_dict = dict()
probleme = list()

for l in link_dict.keys():
    sleep(2)
    try:
        r = get(link_dict[l])
        s = BeautifulSoup(r.content, 'lxml')
        liedtext = s.find_all('div', {'class': 'poem'})
        liedtext = [x.find('p').text.strip() for x in liedtext]
        lyrics_dict[l] = liedtext
    except:
        print('Problem bei', l)
        probleme.append(l)

In [5]:
# Behalten werden bloss die deutschsprachigen Texte. 
for l in lyrics_dict.keys():
    try:
        sp = [detect(x) for x in lyrics_dict[l]]
        n = 0 
        temp_list = list()
        for s in sp:
            if s == 'de':
                temp_list.append(lyrics_dict[l][n])
            n += 1
        if len(temp_list) > 0:
            lyrics_dict[l] = temp_list
    except:
        pass

In [6]:
# Verweise aus der Wikipedia, Strophen-Angaben etc. werden entfernt.
for l in lyrics_dict.keys():
    t = '\n'.join(lyrics_dict[l])
    t = re.sub('Strophe\w\d', '', t)
    t = re.sub('\(\d{1,2}\)', '', t)
    t = re.sub('\[\d{1,2}\]', '', t)
    lyrics_dict[l] = t

In [7]:
# Die einzelnen Texte werden abgespeichert. 
for l in lyrics_dict.keys():
    with open('daten/' + l + '.txt', 'w') as f:
        f.write(lyrics_dict[l])

In [18]:
# Die Texte werden manuell bearbeitet. 
alle_länder = list(lyrics_dict.keys())
#ok = list()
for l in [x for x in alle_länder if x not in ok]:
    system('google-chrome ' + link_dict[l] + ' &')
    system('gedit daten/' + l + '.txt')
    ok.append(l)

### Eine Tabelle mit den Verweisen zu den Dateien wird kreiert

In [174]:
# Eine Tabelle mit den Infos zu den Daten wird kreiert. 
df = pd.DataFrame.from_dict(link_dict, orient='index')
df.columns = ['link_wikipedia']
df['text'] = df.index + '.txt'

In [175]:
# Die Englische Länderbezeichnung wird hinzugefügt. 
df_ländernamen = pd.read_html('https://www.101languages.net/german/country-names-german/')[0]
df = pd.merge(df.reset_index(), df_ländernamen, left_on='index', right_on='German', how='left').set_index('index')

In [176]:
# Der Verweis zur (wahrscheinlich entsprechenden) Midi-Datei wird angefügt. 
f = listdir('midi/')

temp_dict = dict()
for index, row in df.iterrows():
    try:
        temp_dict[index] = process.extractOne(row['English'], [x.replace('.mid', '').capitalize() for x in f])
    except:
        pass

temp_dict = {key:val for key, val in temp_dict.items() if val[1] > 85}
df = pd.merge(df, pd.DataFrame.from_dict(temp_dict, orient='index'), left_index=True, right_index=True, how='left')

In [177]:
# Der Verweis zur (wahrscheinlich entsprechenden) Flagge wird angefügt. 
f = listdir('flaggen/')

temp_dict = dict()
for index, row in df.iterrows():
    try:
        temp_dict[index] = process.extractOne(row['English'], [x.replace('_flag.png', '') for x in f])
    except:
        pass
temp_dict = {key:val for key, val in temp_dict.items() if val[1] > 85}
df = pd.merge(df, pd.DataFrame.from_dict(temp_dict, orient='index'), left_index=True, right_index=True, how='left')

In [178]:
df.columns = ['wikipedia', 'text', 'englisch', 'deutsch', 'midi', 'midi_wahrscheinlichkeit', 'flagge', 'flagge_wahrscheinlichkeit']
df = df[['wikipedia', 'text', 'midi', 'flagge']]
df['flagge'] = df['flagge'].apply(lambda x: str(x).lower() + '_flag.png')
df['midi'] = df['midi'].apply(lambda x: str(x).lower() + '.mid')
df.to_csv('länderübersicht.csv')

In [180]:
!libreoffice länderübersicht.csv

javaldx: Could not find a Java Runtime Environment!
Please ensure that a JVM and the package libreoffice-java-common
is installed.
If it is already installed then try removing ~/.config/libreoffice/4/user/config/javasettings_Linux_*.xml


In [213]:
# Im Midi-Datensatz fehlen einige Länder. Deshalb wird ein zweiter Datensatz hinzugefügt. 
midi_links = dict()
for n in range(1,3):
    r = get('https://www.midiworld.com/search/' + str(n) + '/?q=national%20anthems')
    s = BeautifulSoup(r.content, 'lxml')
    li = s.find_all('ul')[1].find_all('li')
    c = re.search('(.*) \(National Anthems\)', li[0].text).group(1)
    print(c)
    l = li[0].find('a')['href']
    midi_links[c] = l

Afghanistan
Croatia


In [214]:
midi_links

{'Afghanistan': 'https://www.midiworld.com/download/4056',
 'Croatia': 'https://www.midiworld.com/download/4086'}

In [212]:
li[0].find('a')['href']

'https://www.midiworld.com/download/4086'